# Week 9 [Iterators, the `with` statement]

[**Yield expressions**](https://docs.python.org/3/reference/expressions.html#yield-expressions)



## Iterators

    
Example of linked list

In [4]:
class Node:
    def __init__(self, value, nxt=None):
        self.value = value
        self.nxt = nxt

    def get_value(self):
        return self.value

    def get_next(self):
        return self.nxt


class LinkedLiset:
    def __init__(self):
        self.start = None
        self.length = 0
        self.last = None

    def add(self, value):
        elem = Node(value)
        if self.start is None:
            self.start = elem
            self.last = elem
        else:
            self.last.nxt = elem
            self.last = elem
        self.length += 1

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        if idx >= self.length:
            raise IndexError("Index out of range")
        current = self.start
        for i in range(idx):
            current = current.get_next()
        current.get_value()

    def __iter__(self):
        self.__curr = self.start
        return self

    def __next__(self):
        if self.__curr is None:
            raise StopIteration()
        val = self.__curr.get_value()
        self.__curr = self.__curr.get_next()
        return val

In [5]:
lst = LinkedLiset()
for i in range(10):
    lst.add(i*i)
    
for i in lst:
    print(i)

0
1
4
9
16
25
36
49
64
81


### 1

> (2) Реализуйте класс `BinTree` двоичного дерева, итерирование по которму происходит в порядке обхода в глубину.

[**Depth first search of binary tree**](https://en.wikipedia.org/wiki/Tree_traversal#Depth-first_search_of_binary_tree)

this [repo](https://github.com/joowani/binarytree)

In [5]:
from binarytree import tree, Node

# Generate a random binary tree and return its root node
my_tree = tree(height=4, is_perfect=False)
print(my_tree)
print(f'\nIn-order:\n\n{my_tree.inorder}\n')


       ______________25____________
      /                            \
  ___30________           __________19_____
 /             \         /                 \
2            ___18      17___               27___
 \          /     \          \             /     \
  26      _23      3         _12         _5      _15
         /   \              /   \       /       /
        29    7            20    21    24      10


In-order:

[Node(2), Node(26), Node(30), Node(29), Node(23), Node(7), Node(18), Node(3), Node(25), Node(17), Node(20), Node(12), Node(21), Node(19), Node(24), Node(5), Node(27), Node(10), Node(15)]



In [1]:
from binarytree_fucked import tree as fucked_tree
from binarytree_fucked import Node as fucked_Node

In [2]:
my_fucked_tree = fucked_tree(height=4, is_perfect=False)
print(my_fucked_tree)

# Now it is iterable
dfs = [node for node in my_fucked_tree]
print(f'In-order:\n\n {dfs}')


                   ______________17__________
                  /                          \
          _______2______                 _____1______
         /              \               /            \
    ____30             __14__         _29            _27__
   /      \           /      \       /   \          /     \
  19       24        5        16    25    9       _21      13
 /  \        \      / \      /             \     /        /  \
7    18       22   3   4    6               0   23       8    10

In-order:

 [Node(7), Node(19), Node(18), Node(30), Node(24), Node(22), Node(2), Node(3), Node(5), Node(4), Node(14), Node(6), Node(16), Node(17), Node(25), Node(29), Node(9), Node(0), Node(1), Node(23), Node(21), Node(27), Node(8), Node(13), Node(10)]


    
### 2
    
> (3) Скачайте архив, и положите его в папку с неделей 9.
Создаейте класс `TextLoader`, который принимает в инициализаторе адрес архива.
Затем добейтесь от класса следующего поведения:
    - При инициализации объекта (т.е. в методе `__init__`) этот класс должен разархивировать архив в какую-либо папку (имя неважно);
    - Метод `__len__` должен возвращать количество текстов в папке;
    - Метод `__next__`, который позволяет итерироваться по распакованным текстам, должен возвращать объект файла (тот, что возвращает `open()`);
    - При итерировании возвращаются нормализованные тексты, т.е. приводятся к нижнему регистру и убираются знаки препинания.

> Обратите внимание, что метод `__len__` не должен считывать все файлы.
Также нельзя читать все файлы за раз - нужно это делать по мере необходимости.

Docs: [**requests**](https://requests.readthedocs.io/en/master/) | 
[**os**](https://docs.python.org/3/library/os.html) | 
[**zipfile**](https://docs.python.org/3/library/zipfile.html) |
[**pathlib**](https://docs.python.org/3/library/pathlib.html) |
[**re**](https://docs.python.org/3/library/re.html) | 
[**shutil**](https://docs.python.org/3/library/shutil.html)

**See task_2.py**

---
    
## Coroutines  
    

### 3 

> (2) От некоторого устройства в режиме реального времени приходят данные.
Необходимо написать сопрограмму, которая вычисляет среднее, дисперсию, а также количество элементов в переданном наборе данных с устройства.
Результаты работы сопрограмма должна выдавать при отправке соответствующих сигналов.

In [10]:
import numpy as np


class PrintVariance(Exception):
    pass


class PrintMean(Exception):
    pass


class PrintCount(Exception):
    pass


def server_coroutine():
    print("Starting coroutine")
    aggregator = []
    try:
        while True:
            try:
                to_add = yield
                aggregator.append(to_add)
            except PrintVariance:
                yield np.var(aggregator)
            except PrintMean:
                yield np.mean(aggregator)
            except PrintCount:
                yield len(aggregator)
    finally:
        print("Stop coroutine")


coroutine = server_coroutine()
next(coroutine)
for i in range(12):
    coroutine.send(i)
    if i % 2 == 0:
        print("Current variance:", coroutine.throw(PrintVariance))
        next(coroutine)
    if i % 3 == 0:
        print("Current mean:", coroutine.throw(PrintMean))
        next(coroutine)
    if i % 4 == 0:
        print("Current count:", coroutine.throw(PrintCount))
        next(coroutine)
        

coroutine.close()

Starting coroutine
Current variance: 0.0
Current mean: 0.0
Current count: 1
Current variance: 0.6666666666666666
Current mean: 1.5
Current variance: 2.0
Current count: 5
Current variance: 4.0
Current mean: 3.0
Current variance: 6.666666666666667
Current count: 9
Current mean: 4.5
Current variance: 10.0
Stop coroutine


##  `yield from`

In [2]:
def generator1():
    for i in range(5):
        yield f"Generator 1: {i}"
        
def generator2():
    for i in range(5):
        yield f"Generator 2: {i}"
        
def generator():
    yield from generator1()
    yield from generator2()
    
for i in generator():
    print(i)

Generator 1: 0
Generator 1: 1
Generator 1: 2
Generator 1: 3
Generator 1: 4
Generator 2: 0
Generator 2: 1
Generator 2: 2
Generator 2: 3
Generator 2: 4


In [3]:
class Terminate(Exception):
    pass

def inner_coroutine():
    print("Inner coroutine started")
    try:
        while True:
            try:
                x = yield
                print(f"Inner: {x}")
            except Terminate:
                break
    finally:
        print("Inner coroutine finished")
    
def outer_coroutine():
    print("Outer coroutine started")
    try:
        x = yield
        print(f"Outer: {x}")
        x = yield
        print(f"Outer: {x}")
        
        yield from inner_coroutine()
        
        x = yield
        print(f"Outer: {x}")
    finally:
        print("Outer coroutine finished")

In [4]:
try:
    coroutine = outer_coroutine()
    next(coroutine)
    coroutine.send(1)
    coroutine.send(2)
    coroutine.send(3)
    coroutine.send(4)
    coroutine.send(5)
    coroutine.throw(Terminate)
    coroutine.send(6)
except:
    pass

Outer coroutine started
Outer: 1
Outer: 2
Inner coroutine started
Inner: 3
Inner: 4
Inner: 5
Inner coroutine finished
Outer: 6
Outer coroutine finished



### 4    
    
> (4) Представьте, что у вас настроено взаимодействие с сервером, от которого приходят пакеты, содержащие сообщения от различных клиентов. Обработка каждого из клиентов должна идти в отдельном потоке.
Реализуйте:
> 1. Корутину `connect_user`, которая принимает данные авторизации от пользователя, открывает файл с расширением .txt и создает на его основе корутину `write_to_file`.
2. Корутину `write_to_file(f_obj)`, которая записывает переданное планировщиком задач сообщение пользователя в файловый объект, переданный в качестве аргумента при генерации. 
    Также корутина принимает и обрабатывает сигнал об окончании соединения и выходит из сопрограммы.
3. Планировщик задач, распределяющий задачи по сопроцессам на каждого пользователя.

In [4]:
pass; 'use acync/await instead?';

---

## The `with` statement
See [**PEP 343**](https://www.python.org/dev/peps/pep-0343) -- The "with" Statement  | [**contextlib**](https://docs.python.org/3/library/contextlib.html) - Utilities for with-statement contexts

Syntax:

```python
with EXPR as VAR:
    BLOCK
```

The "as VAR" part is optional.
The translation of the above statement is:

```python
mgr = (EXPR)
exit = type(mgr).__exit__  # Not calling it yet
value = type(mgr).__enter__(mgr)
exc = True
try:
    try:
        VAR = value  # Only if "as VAR" is present
        BLOCK
    except:
        # The exceptional case is handled here
        exc = False
        if not exit(mgr, *sys.exc_info()):
            raise
        # The exception is swallowed if exit() returns true
finally:
    # The normal and non-local-goto cases are handled here
    if exc:
        exit(mgr, None, None, None)
```

Here, the lowercase variables (mgr, exit, value, exc) are internal variables and not accessible to the user; they will most likely be implemented as special registers or stack positions.

**Example of class with `__enter__` and `__exit__` methods:**

In [60]:
class DatabaseConnection(object):

    def __init__(self, dbsocket):
        self.dbsocket = dbsocket
    
    def __enter__(self):
        # make a database connection and return it
        
        # let's pretend our database is a file temp.txt
        self.f = open(self.dbsocket, 'r')
        
        return self.f

    def __exit__(self, exc_type, exc_val, exc_tb):
        # make sure the dbconnection (file) gets closed
        self.f.close()

In [61]:
with DatabaseConnection('fucker.txt') as mydbconn:
    print(mydbconn.read())

hey fucker


### Generator Decorator

In [71]:
class GeneratorContextManager(object):

    def __init__(self, gen):
        self.gen = gen

    def __enter__(self):
        try:
            return self.gen.__next__()
        except StopIteration:
            raise RuntimeError('generator didn\'t yield')

    def __exit__(self, type, value, traceback):
        if type is None:
            try:
                self.gen.__next__()
            except StopIteration:
                return
            else:
                raise RuntimeError('generator didn\'t stop')
        else:
            try:
                self.gen.throw(type, value, traceback)
                raise RuntimeError('generator didn\'t stop after throw()')
            except StopIteration:
                return True
            except:
                # only re-raise if it's *not* the exception that was
                # passed to throw(), because __exit__() must not raise
                # an exception unless __exit__() itself failed.  But
                # throw() has to raise the exception to signal
                # propagation, so this fixes the impedance mismatch
                # between the throw() protocol and the __exit__()
                # protocol.
                #
                if sys.exc_info()[1] is not value:
                    raise


def pep343contextmanager(func):
    def helper(*args, **kwds):
        return GeneratorContextManager(func(*args, **kwds))
    return helper

This decorator could be used as follows:

In [78]:
@pep343contextmanager
def opening(filename):
    f = open(filename)  # IOError is untouched by GeneratorContext
    try:
        yield f
    finally:
        f.close()  # Ditto for errors here (however unlikely)
        

with opening('fucker.txt') as f:
    print(f.read())

hey fucker


A more robust version is provided in the standard library:

In [76]:
from contextlib import contextmanager

# A template for ensuring that a lock, acquired at the start of a block, is released when the block is left:
@contextmanager
def locked(lock):
    lock.acquire()
    try:
        yield
    finally:
        lock.release()

### More examples

A template for committing or rolling back a database transaction:

In [79]:
@contextmanager
def transaction(db):
    db.begin()
    try:
        yield None
    except:
        db.rollback()
        raise
    else:
        db.commit()